#### Insturctions on how to prepare the exmaple data used in the notebooks
To prepare the example data yourself, you need to first download the raw ERA5 data (2 meter temperature) and upscale the data to daily timescale and regrid it to 2 degree. These steps can be done with `era5cli` and `cdo`. Check the [README.md](./README.md) for more information.

Afterward, please follow this notebook to prepare the clustered 2 meter temperature dataset.

In [1]:
import numpy as np
from pathlib import Path
import xarray as xr

Load your dataset

In [23]:
path_to_data = "./"
t2m_raw = xr.load_dataset(Path(path_to_data, "t2m_1959-2021_1_12_daily_2.0deg.nc"))
t2m_raw

<xarray.Dataset>
Dimensions:    (time: 23011, longitude: 180, latitude: 91)
Coordinates:
  * time       (time) datetime64[ns] 1959-01-01 1959-01-02 ... 2021-12-31
  * longitude  (longitude) float32 -180.0 -178.0 -176.0 ... 174.0 176.0 178.0
  * latitude   (latitude) float32 90.0 88.0 86.0 84.0 ... -86.0 -88.0 -90.0
Data variables:
    t2m        (time, latitude, longitude) float32 240.1 240.1 ... 250.3 250.3
Attributes:
    CDI:          Climate Data Interface version 1.9.10 (https://mpimet.mpg.d...
    Conventions:  CF-1.6
    history:      Mon Aug 08 11:48:03 2022: cdo -b 32 settime,00:00 -daymean ...
    frequency:    day
    CDO:          Climate Data Operators version 1.9.10 (https://mpimet.mpg.d...

Sort by latitudes and longitudes to make the coordinate monotonically increase

In [3]:
t2m_raw = t2m_raw.assign_coords(longitude=((t2m_raw.longitude + 360) % 360))
t2m_raw = t2m_raw.sortby('longitude') # otherwise slicing does not work due to jump in data
t2m_raw = t2m_raw.sortby('latitude')
t2m_raw

<xarray.Dataset>
Dimensions:    (time: 23011, longitude: 180, latitude: 91)
Coordinates:
  * time       (time) datetime64[ns] 1959-01-01 1959-01-02 ... 2021-12-31
  * longitude  (longitude) float32 0.0 2.0 4.0 6.0 ... 352.0 354.0 356.0 358.0
  * latitude   (latitude) float32 -90.0 -88.0 -86.0 -84.0 ... 86.0 88.0 90.0
Data variables:
    t2m        (time, latitude, longitude) float32 242.9 242.9 ... 250.4 250.4
Attributes:
    CDI:          Climate Data Interface version 1.9.10 (https://mpimet.mpg.d...
    Conventions:  CF-1.6
    history:      Mon Aug 08 11:48:03 2022: cdo -b 32 settime,00:00 -daymean ...
    frequency:    day
    CDO:          Climate Data Operators version 1.9.10 (https://mpimet.mpg.d...

Load clusters mask

In [4]:
data_folder = '~/AI4S2S/data'
cluster_mask = xr.open_dataset(Path(data_folder,'tf5_nc5_dendo_80d77.nc'))
cluster_mask

<xarray.Dataset>
Dimensions:      (latitude: 41, longitude: 76, cluster: 6, time: 14600)
Coordinates:
    tfreq        int64 ...
    n_clusters   int64 ...
  * longitude    (longitude) float64 225.0 226.0 227.0 ... 298.0 299.0 300.0
  * latitude     (latitude) float32 70.0 69.0 68.0 67.0 ... 33.0 32.0 31.0 30.0
    mask         (latitude, longitude) int8 ...
  * cluster      (cluster) int64 1 2 3 4 5 6
  * time         (time) datetime64[ns] 1979-01-01 1979-01-02 ... 2018-12-31
Data variables:
    xrclustered  (latitude, longitude) float32 ...
    ts           (cluster, time) float64 ...
    q75tail      (cluster, time) float64 ...

In [5]:
cluster_mask_2_deg = cluster_mask["xrclustered"][::2, 1::2]
cluster_mask_2_deg = cluster_mask_2_deg.drop("tfreq")
cluster_mask_2_deg = cluster_mask_2_deg.sortby('latitude')
cluster_mask_2_deg

<xarray.DataArray 'xrclustered' (latitude: 21, longitude: 38)>
[798 values with dtype=float32]
Coordinates:
    n_clusters  int64 ...
  * longitude   (longitude) float64 226.0 228.0 230.0 ... 296.0 298.0 300.0
  * latitude    (latitude) float32 30.0 32.0 34.0 36.0 ... 64.0 66.0 68.0 70.0
    mask        (latitude, longitude) int8 ...
Attributes:
    long_name:     Maximum temperature at 2 metres since previous post-proces...
    units:         K
    is_DataArray:  1
    method:        AgglomerativeClustering
    kwrgs:         {'q': 66, 'n_clusters': [2, 3, 4, 5, 6, 7, 8], 'affinity':...
    target:        mx2t_exceedances_of_66th_percentile
    hash:          80d77

Match the domain size of input data as the mask

In [6]:
def match_coords_xarrays(wanted_coords_arr, to_match):
    dlon = float(to_match.longitude[:2].diff('longitude'))
    dlat = float(to_match.latitude[:2].diff('latitude'))
    lonmin = wanted_coords_arr.longitude.min()
    lonmax = wanted_coords_arr.longitude.max()
    latmin = wanted_coords_arr.latitude.min()
    latmax = wanted_coords_arr.latitude.max()
    return to_match.sel(longitude=np.arange(lonmin, lonmax+dlon,dlon),
                       latitude=np.arange(latmin, latmax+dlat,dlat),
                       method='nearest')

In [7]:
# matching domain of xrclustered DataArray
t2m_raw = match_coords_xarrays(cluster_mask_2_deg, t2m_raw)

Take the mean for each cluster using the cluster masks

In [8]:
list_data_arrays = []

for i in range(1, cluster_mask["n_clusters"].values + 1):
    print(f"Process cluster number {i}")
    t2m_cluster = (
        t2m_raw["t2m"]
        .where(cluster_mask_2_deg.where(cluster_mask_2_deg == i).fillna(False), drop=False)
        .mean(dim=["latitude", "longitude"])
    )
    t2m_cluster.expand_dims({"cluster": 1}, axis=0)
    t2m_cluster["cluster"] = i
    list_data_arrays.append(t2m_cluster)

Process cluster number 1
Process cluster number 2
Process cluster number 3
Process cluster number 4
Process cluster number 5
Process cluster number 6


In [9]:
t2m_target = xr.concat(list_data_arrays, dim='n_cluster')

In [10]:
t2m_target

<xarray.DataArray 't2m' (n_cluster: 6, time: 23011)>
array([[248.2771 , 240.65463, 240.35861, ..., 244.8182 , 246.06013,
        244.44157],
       [263.5156 , 251.37112, 245.67123, ..., 249.24133, 248.64215,
        247.91904],
       [272.07837, 272.4065 , 267.74524, ..., 277.61002, 277.49585,
        279.68536],
       [255.5705 , 259.87503, 262.5816 , ..., 258.99667, 261.77206,
        260.53543],
       [271.82706, 266.94107, 260.25592, ..., 264.7034 , 266.73074,
        267.99063],
       [272.51218, 277.17313, 267.65158, ..., 286.29132, 284.082  ,
        286.3894 ]], dtype=float32)
Coordinates:
  * time        (time) datetime64[ns] 1959-01-01 1959-01-02 ... 2021-12-31
    n_clusters  int64 6
    cluster     (n_cluster) int64 1 2 3 4 5 6
Dimensions without coordinates: n_cluster

Save the cluster data and cluster mask into a single dataset

In [18]:
t2m_target_dataset = xr.Dataset({"t2m": t2m_target, "xrcluster": cluster_mask_2_deg})
t2m_target_dataset

<xarray.Dataset>
Dimensions:     (time: 23011, n_cluster: 6, longitude: 38, latitude: 21)
Coordinates:
  * time        (time) datetime64[ns] 1959-01-01 1959-01-02 ... 2021-12-31
    n_clusters  int64 6
    cluster     (n_cluster) int64 1 2 3 4 5 6
  * longitude   (longitude) float64 226.0 228.0 230.0 ... 296.0 298.0 300.0
  * latitude    (latitude) float32 30.0 32.0 34.0 36.0 ... 64.0 66.0 68.0 70.0
    mask        (latitude, longitude) int8 ...
Dimensions without coordinates: n_cluster
Data variables:
    t2m         (n_cluster, time) float32 248.3 240.7 240.4 ... 284.1 286.4
    xrcluster   (latitude, longitude) float32 nan nan nan nan ... nan nan nan

Add attributes to variables and the dataset

In [22]:
# add attribute for t2m
t2m_target_dataset["t2m"] = t2m_target_dataset["t2m"].assign_attrs(
    {"long_name": "clustered 2 meter temperature", "units": "K"}
)

# add attribute to cluster masks
t2m_target_dataset["xrcluster"].attrs = {"long_name": "Cluster masks for t2m",
                                         "units": 1,
                                         "method": "AgglomerativeClustering",
                                         "kwrgs": "{'q': 66, 'n_clusters': [2, 3, 4, 5, 6, 7, 8], 'affinity': 'jaccard', 'linkage': 'average'}",
                                         "target": "mx2t_exceedances_of_66th_percentile"}

# add global attribute to the dataset
t2m_target_dataset = t2m_target_dataset.assign_attrs(
    {
        "history": "The dataset contains 2 meter temperature clustered using Agglomerative Clustering approach w.r.t rainfall depth.",
        "source": "ERA5",
    }
)

t2m_target_dataset

<xarray.Dataset>
Dimensions:     (time: 23011, n_cluster: 6, longitude: 38, latitude: 21)
Coordinates:
  * time        (time) datetime64[ns] 1959-01-01 1959-01-02 ... 2021-12-31
    n_clusters  int64 6
    cluster     (n_cluster) int64 1 2 3 4 5 6
  * longitude   (longitude) float64 226.0 228.0 230.0 ... 296.0 298.0 300.0
  * latitude    (latitude) float32 30.0 32.0 34.0 36.0 ... 64.0 66.0 68.0 70.0
    mask        (latitude, longitude) int8 ...
Dimensions without coordinates: n_cluster
Data variables:
    t2m         (n_cluster, time) float32 248.3 240.7 240.4 ... 284.1 286.4
    xrcluster   (latitude, longitude) float32 nan nan nan nan ... nan nan nan
Attributes:
    history:  The dataset contains 2 meter temperature clustered using Agglom...
    source:   ERA5

Export processed data as netcdf4 file

In [29]:
t2m_target_dataset.to_netcdf(Path(path_to_data, "t2m_daily_1950-2021_2deg_clustered_226_300E_30_70N.nc"))

In [11]:
path_to_data = "./"
sst = xr.load_dataset("./sst_daily_1950-2021_5deg_Pacific_175_240E_25_50N.nc")
sst

<xarray.Dataset>
Dimensions:    (time: 23011, longitude: 13, latitude: 5)
Coordinates:
  * time       (time) datetime64[ns] 1959-01-01 1959-01-02 ... 2021-12-31
  * longitude  (longitude) float64 177.5 182.5 187.5 192.5 ... 227.5 232.5 237.5
  * latitude   (latitude) float64 47.5 42.5 37.5 32.5 27.5
Data variables:
    sst        (time, latitude, longitude) float32 277.7 277.9 ... 291.9 291.5
Attributes:
    CDI:          Climate Data Interface version 2.2.1 (https://mpimet.mpg.de...
    Conventions:  CF-1.6
    history:      Wed Jun 14 17:26:04 2023: cdo remapbil,/Users/semv/surfdriv...
    CDO:          Climate Data Operators version 2.2.0 (https://mpimet.mpg.de...